# Latar Belakang Masalah

Dalam era digital, e-commerce telah menjadi sektor vital dengan pertumbuhan pesat yang menghasilkan volume data besar. Mengelola dan menganalisis data ini adalah tantangan namun juga peluang untuk meningkatkan efisiensi, memahami perilaku pelanggan, dan meningkatkan pengalaman pengguna.

Dataset E-commerce Order and Product Details menyediakan informasi lengkap terkait pesanan, item dalam pesanan, pelanggan, pembayaran, dan produk pada platform e-commerce. Dataset ini disusun dalam beberapa tabel yang masing-masing berisi informasi spesifik.

Analisis data ini dapat mengungkap wawasan berharga untuk pengambilan keputusan strategis. Misalnya, memahami faktor-faktor penyebab keterlambatan pengiriman dapat meningkatkan proses logistik dan kepuasan pelanggan. Demikian pula, analisis pola pembayaran dan preferensi produk dapat membantu penyesuaian strategi pemasaran.

Pendekatan analitis yang tepat sangat penting untuk memanfaatkan potensi dataset ini dan mendukung pertumbuhan serta kesuksesan platform e-commerce.

# Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import optuna

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

from category_encoders import TargetEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import  XGBClassifier

py.init_notebook_mode(connected=True)

# Data Preprocessing

## Import Data

In [2]:
test_id = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test_order_id.csv")

In [3]:
df_customers = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/train/df_Customers.csv")
df_Orderitems = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/train/df_OrderItems.csv")
df_Orders = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/train/df_Orders.csv")
df_Payments = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/train/df_Payments.csv")
df_Products = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/train/df_Products.csv")

In [4]:
df_customersT = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test/df_Customers.csv")
df_OrderitemsT = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test/df_OrderItems.csv")
df_OrdersT = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test/df_Orders.csv")
df_PaymentsT = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test/df_Payments.csv")
df_ProductsT = pd.read_csv("/Users/faizralvi/Documents/Lomba Intelligo 2023/fix-tahap-1-lomba-data-science-intelligo-id/test/df_Products.csv")

## Menggabungkan Semua Data

In [5]:
df_Products = pd.concat([df_Products, df_ProductsT], ignore_index=True)
df_Products = df_Products.drop_duplicates()

In [6]:
df_Products

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,90K0C1fIyQUf,toys,491.0,19.0,12.0,16.0
1,qejhpMGGVcsl,watches_gifts,440.0,18.0,14.0,17.0
2,qUS5d2pEAyxJ,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,639iGvMyv0De,toys,1450.0,68.0,3.0,48.0
4,1lycYGcsic2F,toys,300.0,17.0,4.0,12.0
...,...,...,...,...,...,...
127545,0a4ravzgJwZa,toys,200.0,16.0,11.0,11.0
127573,rNfpKHAxqqjp,toys,3450.0,30.0,40.0,11.0
127575,XuAmpXlawktw,NaN,700.0,24.0,4.0,15.0
127583,Siea5ovIiWWX,toys,900.0,30.0,30.0,30.0


In [7]:
df = df_Orderitems.merge(df_Orders, on='order_id', how='left')
df = df.merge(df_Payments, on='order_id', how='left')
df = df.merge(df_customers, on='customer_id', how='left')
df = df.merge(df_Products, on='product_id', how='left')


In [8]:
test = df_OrderitemsT.merge(df_OrdersT, on='order_id', how='left')
test = test.merge(df_PaymentsT, on='order_id', how='left')
test = test.merge(df_customersT, on='customer_id', how='left')
test = test.merge(df_Products, on='product_id', how='left')


## Feature Enginering

In [9]:
# Menghilangkan baris yang tidak memiliki kolom order_delivered_timestamp, dikarenakan tidak dapat mengetahui keterlambatan
df = df.dropna(subset=['order_delivered_timestamp'])

### Membuat Kolom Keterlambatan kolom yang di perlukan

In [10]:
# Buat DataFrame Anda dengan dua kolom 'order_estimated_delivery_date' dan 'order_delivered_timestamp'

# Konversi kolom-kolom ke tipe data datetime
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['order_delivered_timestamp'] = pd.to_datetime(df['order_delivered_timestamp'])

# Hitung selisih dalam hari antara dua tanggal dan simpan hasilnya dalam kolom baru
df['selisih_hari'] = (df['order_estimated_delivery_date'] - df['order_delivered_timestamp']).dt.days

for i, row in df.iterrows():
    selisih_hari = row['selisih_hari']
    if selisih_hari < 0:
        df.at[i, 'is_late'] = 1
    else:
        df.at[i, 'is_late'] = 0


In [11]:
df['volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']
test['volume'] = test['product_length_cm'] * test['product_height_cm'] * test['product_width_cm']

### melakukan preprocessing tambahan

In [12]:
# menghilangkan unique value
df = df.drop(columns=['order_id','customer_id'], axis=1)
# lalu menghilangkan duplikat
df = df.drop_duplicates()

### Membuat Data Untuk Model

In [13]:
# Define columns to drop
columns_to_drop = [ 'product_id', 'seller_id', 
                   'order_purchase_timestamp', 'order_approved_at', 'customer_zip_code_prefix']

# Additional columns specific to the df DataFrame
additional_columns_to_drop_df = ['selisih_hari', 'order_status', 
                                 'order_delivered_timestamp', 'order_estimated_delivery_date']

# Drop columns from df
df_train = df.drop(columns=columns_to_drop + additional_columns_to_drop_df, axis=1)

# Drop columns from test
test = test.drop(columns=columns_to_drop + ['order_id','customer_id'], axis=1)

print("----- Columns Dropped -----")


----- Columns Dropped -----


# EDA (Exploratory Data Analysis)

Untuk mendapatkan insight yang lebih mendalam tentang penyebab keterlambatan pengiriman barang, kita bisa melakukan beberapa langkah analisis eksploratif data (EDA) yang lebih terperinci. 

## Distribusi Data

Langkah pertama dalam analisis ini adalah memeriksa distribusi data keterlambatan pengiriman. Grafik awal menunjukkan bahwa jumlah pengiriman yang tepat waktu (on-time) secara signifikan lebih tinggi dibandingkan dengan jumlah pengiriman yang terlambat. Hal ini mengindikasikan bahwa meskipun ada beberapa kasus keterlambatan, mayoritas pengiriman berlangsung sesuai dengan jadwal yang ditentukan.

In [14]:
data = [go.Bar(
            x = df["is_late"].value_counts().index.values,
            y = df["is_late"].value_counts().values,
            text=['On-time distribution', 'Late distribution'],
            marker=dict(color=['blue', 'green'])
    )]

layout = go.Layout(
    title='Distribusi Keterlambatan',
    width=600,  # Lebar grafik dalam piksel
    height=400  # Tinggi grafik dalam piksel
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

## Mencari Penyebab Keterlambatan

Langkah pertama dalam analisis ini adalah memeriksa jumlah pengiriman berdasarkan tanggal persetujuan pesanan. Umumnya, setelah pesanan disetujui oleh penjual, barang akan langsung dikirimkan ke pembeli. Grafik berikut menunjukkan jumlah pesanan yang dikonfirmasi pada tanggal 24 April 2018, yang mencapai 770 pesanan.

In [15]:
# Langkah 1: Mengubah format kolom order_approved_at menjadi datetime
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

# Langkah 2: Mengubah kolom order_approved_at menjadi hanya tanggal
df['order_approved_date'] = df['order_approved_at'].dt.date

# Langkah 3: Menghitung jumlah pengiriman per hari
delivery_counts_daily = df.groupby('order_approved_date').size().reset_index(name='delivery_count')

# Langkah 4: Membuat visualisasi dengan Plotly
fig = px.line(
    delivery_counts_daily,
    x='order_approved_date',
    y='delivery_count',
    title='Jumlah Pengiriman Berdasarkan Tanggal Persetujuan Pesanan',
    labels={'order_approved_date': 'Tanggal Persetujuan Pesanan', 'delivery_count': 'Jumlah Pengiriman'}
)

fig.show()


Selanjutnya, kita memfilter data untuk fokus hanya pada pesanan yang terlambat dan membuat grafik untuk menganalisisnya. Dari grafik tersebut, terlihat bahwa jumlah keterlambatan tertinggi terjadi pada tanggal 24 November 2017, dengan total 119 kasus keterlambatan.

In [16]:

# Langkah 1: Mengubah format kolom order_approved_at menjadi datetime
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

# Langkah 2: Mengubah kolom order_approved_at menjadi hanya tanggal
df['order_approved_date'] = df['order_approved_at'].dt.date

# Langkah 3: Menghitung jumlah keterlambatan per hari berdasarkan tanggal persetujuan
late_counts_daily = df[df['is_late'] == 1].groupby('order_approved_date').size().reset_index(name='late_count')

# Langkah 4: Membuat visualisasi dengan Plotly
fig = px.line(
    late_counts_daily,
    x='order_approved_date',
    y='late_count',
    title='Jumlah Keterlambatan Berdasarkan Tanggal Persetujuan Pesanan',
    labels={'order_approved_date': 'Tanggal Persetujuan Pesanan', 'late_count': 'Jumlah Keterlambatan'}
)

fig.update_traces(line=dict(color='orange'))

fig.show()



Saat kita menganalisis lebih detail pada tanggal 24 November 2017, grafik menunjukkan bahwa tidak ada aktivitas yang signifikan sepanjang hari yang menyebabkan keterlambatan. Hal ini mengindikasikan bahwa penyebab keterlambatan pada tanggal tersebut mungkin tidak terhubung dengan pola harian yang terlihat dalam data.

In [17]:

# Langkah 1: Mengubah format kolom order_approved_at menjadi datetime
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

# Langkah 2: Filter data untuk tanggal 9 Februari 2017
df_nov_24_2017 = df[df['order_approved_at'].dt.date == pd.to_datetime('2017-11-24').date()]

# Langkah 3: Menghitung jumlah keterlambatan untuk setiap waktu persetujuan pesanan pada tanggal tersebut
late_counts = df_nov_24_2017[df_nov_24_2017['is_late'] == 1].groupby('order_approved_at').size().reset_index(name='late_count')

# Langkah 4: Membuat visualisasi dengan Plotly
fig = px.line(
    late_counts,
    x='order_approved_at',
    y='late_count',
    title='Jumlah Keterlambatan Berdasarkan Waktu Persetujuan Pesanan pada 24 November 2017',
    labels={'order_approved_at': 'Waktu Persetujuan Pesanan', 'late_count': 'Jumlah Keterlambatan'}
)

fig.update_traces(line=dict(color='red'))

fig.show()


Selanjutnya, mari kita lihat distribusi jumlah pengiriman berdasarkan kota pada tanggal 24 November 2017. Hasil analisis menunjukkan bahwa pengiriman terbanyak pada hari tersebut bertujuan ke kota São Paulo. Hal ini tercermin dari data yang menunjukkan bahwa São Paulo menerima jumlah pengiriman terbanyak dibandingkan kota lainnya pada hari itu.

In [18]:

# Langkah 3: Menghitung jumlah keterlambatan per kota
late_counts_city = df_nov_24_2017['customer_city'].value_counts().head(5).reset_index()

# Langkah 4: Mengubah nama kolom untuk lebih mudah dibaca
late_counts_city.columns = ['customer_city', 'jumlah_pengiriman']

# Tampilkan DataFrame
late_counts_city



,customer_city,jumlah_pengiriman
0,sao paulo,79
1,rio de janeiro,55
2,belo horizonte,26
3,porto alegre,10
4,fortaleza,9


Ketika kita memfilter data untuk hanya mencakup pengiriman yang terlambat pada tanggal 24 November 2017, terlihat bahwa meskipun pengiriman terbanyak pada hari itu menuju kota São Paulo, kota Rio de Janeiro mengalami jumlah keterlambatan yang lebih tinggi dibandingkan São Paulo.

Ini menunjukkan bahwa meskipun São Paulo menerima lebih banyak pengiriman, Rio de Janeiro memiliki tingkat keterlambatan yang lebih tinggi pada hari tersebut. Hal ini bisa menunjukkan bahwa faktor-faktor tertentu mungkin mempengaruhi keterlambatan di Rio de Janeiro lebih signifikan dibandingkan dengan São Paulo, meskipun jumlah pengiriman tidak sebanyak di São Paulo.

In [19]:

# Menghitung jumlah keterlambatan per kota
late_city_counts = df_nov_24_2017[df_nov_24_2017['is_late'] == 1]['customer_city'].value_counts()

# Mengambil 5 kota terbesar
top_5_late_cities = late_city_counts.head(5)

# Mengubah ke dalam DataFrame untuk menampilkan tabel
top_5_late_cities_df = top_5_late_cities.reset_index()
top_5_late_cities_df.columns = ['Customer City', 'Late Count']

# Menampilkan tabel

top_5_late_cities_df


,Customer City,Late Count
0,rio de janeiro,17
1,sao paulo,7
2,belo horizonte,4
3,fortaleza,4
4,brasilia,4


Bukti ini dapat diperkuat dari grafik di bawah ini, yang menunjukkan bahwa jumlah keterlambatan di Rio de Janeiro hampir menyamai jumlah keterlambatan di São Paulo. Padahal, jika dilihat dari jumlah pengiriman, rasio pengiriman ke São Paulo jauh lebih tinggi dibandingkan dengan Rio de Janeiro.

Grafik ini menggambarkan bahwa meskipun São Paulo menerima jumlah pengiriman yang lebih besar, keterlambatan di Rio de Janeiro hampir setara dengan keterlambatan di São Paulo. Hal ini menunjukkan bahwa kota dengan jumlah pengiriman lebih banyak tidak selalu memiliki jumlah keterlambatan yang lebih besar secara proporsional.

In [20]:
# Langkah 2: Menghitung jumlah pesanan per kota berdasarkan keterlambatan
city_counts = df.groupby(['customer_city', 'is_late']).size().reset_index(name='count')

# Mengubah format data untuk plot
city_pivot = city_counts.pivot(index='customer_city', columns='is_late', values='count').fillna(0)
city_pivot['late'] = city_pivot[1]
city_pivot['on_time'] = city_pivot[0]
city_pivot['total'] = city_pivot['late'] + city_pivot['on_time']

# Mendapatkan 10 kota teratas berdasarkan jumlah pesanan terlambat
top_10_cities = city_pivot.nlargest(10, 'late').reset_index()

# Mengubah Data untuk Plotly
# Melt data untuk format yang sesuai dengan Plotly
top_10_cities_melted = top_10_cities.melt(id_vars='customer_city', value_vars=['on_time', 'late'], var_name='status', value_name='count')

# Visualisasi dengan Plotly
fig = px.bar(
    top_10_cities_melted,
    y='customer_city',
    x='count',
    color='status',
    title='10 Kota Teratas Berdasarkan Jumlah Pesanan Terlambat',
    labels={'count': 'Jumlah Pesanan', 'customer_city': 'Kota'},
    text='count',
    orientation='h',
    color_discrete_map={'on_time': 'blue', 'late': 'red'}
)

fig.update_layout(barmode='stack')
fig.show()


## Kesimpulan EDA

Analisis data pengiriman dan keterlambatan menunjukkan bahwa meskipun São Paulo menerima jumlah pengiriman yang jauh lebih besar dibandingkan Rio de Janeiro, jumlah keterlambatan di kedua kota tersebut hampir sebanding. Hal ini menyoroti sebuah pola menarik: meskipun São Paulo, sebagai kota dengan volume pengiriman yang lebih tinggi, memiliki tingkat keterlambatan yang relatif lebih rendah dibandingkan dengan Rio de Janeiro.

Kemungkinan Penyebab:

Hasil ini mengindikasikan bahwa keterlambatan pengiriman mungkin dipengaruhi oleh faktor-faktor geografis dan logistik. Misalnya:

- Jarak dan Infrastruktur: Keterlambatan dapat terjadi karena jarak yang lebih jauh atau infrastruktur transportasi yang kurang efisien di beberapa wilayah.

- Kondisi Cuaca: Faktor cuaca yang buruk atau kondisi jalan yang tidak optimal di daerah tertentu dapat memperlambat proses pengiriman.

- Kepadatan Lalu Lintas: Kota dengan kepadatan lalu lintas yang tinggi atau masalah transportasi lainnya dapat mengalami keterlambatan yang lebih sering, meskipun jumlah pengiriman mungkin lebih rendah.

Dengan kata lain, meskipun volume pengiriman di São Paulo jauh lebih besar, faktor-faktor geografis dan infrastruktur di Rio de Janeiro mungkin berkontribusi pada tingkat keterlambatan yang lebih tinggi secara proporsional. Oleh karena itu, perlu dilakukan evaluasi lebih lanjut terhadap infrastruktur logistik dan faktor-faktor eksternal di masing-masing kota untuk mengidentifikasi dan mengatasi penyebab keterlambatan ini secara efektif.

# Modeling

## Encoding Data Categorical

In [21]:

# Assuming df is your DataFrame
df_importance = df_train.copy().drop("is_late", axis=1)
late = df["is_late"]

# Identify categorical columns
categorical = df_importance.select_dtypes("object").columns

# Apply target encoding
target_encoder = TargetEncoder(cols=categorical)

df_importance[categorical] = target_encoder.fit_transform(df_importance[categorical], late)
test[categorical] = target_encoder.transform(test[categorical])

## Input Kolom Kosong

In [22]:

# Membuat objek KNNImputer
imputer = KNNImputer(n_neighbors=3)

# Mengimputasi nilai yang hilang
df_imputed = pd.DataFrame(imputer.fit_transform(df_importance), columns=df_importance.columns)
test_imputed = pd.DataFrame(imputer.transform(test), columns=test.columns)

# Menampilkan DataFrame yang telah diimputasi
df_imputed.head()


,price,shipping_charges,payment_sequential,payment_type,payment_installments,payment_value,customer_city,customer_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,volume
0,223.51,84.65,1.0,0.077634,1.0,259.14,0.043742,0.056628,0.079228,491.0,19.0,12.0,16.0,3648.0
1,170.80,23.79,1.0,0.077634,8.0,382.39,0.310531,0.131728,0.090411,440.0,18.0,14.0,17.0,4284.0
2,64.40,17.38,1.0,0.077634,4.0,249.25,0.067619,0.056628,0.058989,2200.0,16.0,16.0,16.0,4096.0
3,264.50,30.72,1.0,0.077634,2.0,27.79,0.041237,0.055574,0.079228,1450.0,68.0,3.0,48.0,9792.0
4,779.90,30.66,1.0,0.077634,1.0,76.15,0.062837,0.056628,0.079228,300.0,17.0,4.0,12.0,816.0


## Pencarian Model Machine Learning

In [23]:

X = df_imputed
y = late

# Membagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Daftar model untuk dibandingkan
models = {
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Catboost': CatBoostClassifier(random_state=42, verbose=0)
}

# Fungsi untuk mengevaluasi model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, precision, recall, f1

# Membandingkan semua model
results = {}
for name, model in models.items():
    accuracy, precision, recall, f1 = evaluate_model(model, X_train, X_test, y_train, y_test)
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Menampilkan hasil
results_df = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'Model'})



/Users/faizralvi/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [24]:
results_melted = results_df.melt(id_vars='Model', var_name='Metric', value_name='Score')

fig = px.bar(
    results_melted,
    x='Model',
    y='Score',
    color='Metric',
    barmode='group',
    title='Perbandingan Kinerja Model ML',
    labels={'Score': 'Skor', 'Model': 'Model'}
)

fig.show()

## Tuning Parameter

In [25]:

RETRAIN_XGB_MODEL = False

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'random_state': 42,
        'n_jobs': -1
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy_scores = []

    for train_idx, valid_idx in cv.split(X, y):
        X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]
        model = XGBClassifier(**param)
        model.fit(X_train_fold, y_train_fold)
        accuracy = accuracy_score(y_valid_fold, model.predict(X_valid_fold))
        accuracy_scores.append(accuracy)

    return np.mean(accuracy_scores)

study = optuna.create_study(direction='maximize', study_name="xgb_model_training")
if RETRAIN_XGB_MODEL:
    study.optimize(objective, n_trials=200)
    print(f"Best trial average accuracy: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

# Train final model with best parameters
best_params = study.best_params
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# Evaluate model
final_accuracy = accuracy_score(y_test, final_model.predict(X_test))
print(f'Final accuracy on test set: {final_accuracy:.4f}')


[I 2024-08-02 14:28:01,778] A new study created in memory with name: xgb_model_training


ValueError: No trials are completed yet.

## Prediksi Data Baru

In [ ]:
params = {'n_estimators': 204, 'max_depth': 10, 'min_child_weight': 2, 'gamma': 0.43983434675592775, 'learning_rate': 0.06915702878617219, 'subsample': 0.7469030977499815, 'colsample_bytree': 0.8537436283742482}

In [ ]:
model = XGBClassifier(random_state=42, **params)
model.fit(X, y)

predict = model.predict(test_imputed)

In [ ]:
test_id['is_late'] = predict
test_id.head()